In [264]:
import pandas as pd
import csv
from itertools import permutations 
from math import pi,sin,cos,acos,floor

In [265]:
class Aircraft: # Class that contains all aircraft
    
    def __init__(self, model, manufacturer, max_capacity, current_capacity = 0):
        self._model = model # model of the aircraft
        self.manufacturer = manufacturer # manufacturer of the aircraft
        self._max_capacity = max_capacity # max capacity of the aircraft
        self._current_capacity = current_capacity # current capacity of the aircraft
    
    def get_max_capacity(self):
        return self._max_capacity # returns the max distance
    
    def get_maunfacturer(self):
        return self._manufacturer # returns the manufacturer
    
    def get_model(self):
        return self._model # returns the model

    def current_capacity(self):
        return self._current_capacity # returns the current capacity left in the plane
    
    def current_capacity(self, amount):
        if self._max_capacity == self._current_capacity: # This is just a checker.
            return "Sorry, the plane has a full tank"
        elif self._current_capacity + amount > self._max_capacity:
            return "Sorry, you have filled up by too much"
        else:
            self._current_capacity = amount # sets the capacity of the plane

class Airport:
    
    def __init__(self, airport_code, longitude, latitude, exchangeRate):
        self.airport_code = airport_code # airport code of the airport
        self._longitude = longitude # longitude of the airport
        self._latitude = latitude # latitude of the airport
        self._exchange_rate = exchangeRate # currency used by the airport
        
    def get_longitude(self):
        return self._longitude # returns the longitude of the airport
    
    def get_latitude(self):
        return self._latitude # returns the latitude of the airport
    
    def get_exchange_rate(self):
        return self._exchange_rate # returns the currency used by the airport
    
    def set_exchange_rate(self, exchangeRate):
        self._exchange_rate = exchangeRate # sets the currency of the airport
        

In [241]:
#Optimum route will take in the list of destinations and the plane model and create a list of dictionary
#of airport objects. Each object contains airport codes, latitude, longitude, and the exchange rate to Euro.
#The key for the dictionary of Airport objects is the airport code, and of course the value is the object

#time complexity: 2n^2 + 4n ----> o(n^2)
def optimum_route(listx): 

    airport_list = [] # creates a new list

    with open('airport.csv', newline='', encoding="utf8") as airport_file: # opens the csv file
        reader = csv.reader(airport_file) # reads the cotents to a variable
        next(reader, None) # returns none at the end of the file
        for airport in reader: # iterates through the reader
            if airport[4] in listx:
                airport_code = airport[4] # assigns variable
                country_name = airport[3] # assigns variable
                longitude = airport[6] # assigns variable
                latitude = airport[7] # assigns variable
                templist = [airport_code, country_name, longitude, latitude]
                airport_list.append(templist)       
    
    country_currency_list = [] # creates a new list

    with open('countrycurrency.csv', newline='', encoding="utf8") as countrycurrency_file: # opens the csv file
        reader = csv.reader(countrycurrency_file) # reads the cotents to a variable
        next(reader, None) # returns none at the end of the file
        for country in reader: # iterates through the reader
            temp_list = [] # temp list created
            temp_list.append(country[0]) # appends value to list
            temp_list.append(country[14]) # appends value to list
            country_currency_list.append(temp_list) # appends temp list to the main list
    
    currency_list = [] # creates a new list

    with open('currencyrates.csv', newline='', encoding="utf8") as currencyrates_file: # opens the csv file
        reader = csv.reader(currencyrates_file) # reads the cotents to a variable
        next(reader, None) # returns none at the end of the file
        for currency in reader: # iterates through the reader
            temp_list = [] # temp list created
            temp_list.append(currency[1]) # appends value to list
            temp_list.append(currency[2]) # appends value to list
            currency_list.append(temp_list) # appends temp list to the main list
            
    
    final_list = []
    for i in country_currency_list:
        for x in currency_list:
            if i[1] == x[0]:
                templist = [i[0], x[1]]
                final_list.append(templist)

    x = 0
    i = 0
    while x < len(airport_list):
        while i < len(final_list):
            if airport_list[x][1] == final_list[i][0]:
                airport_list[x].extend(final_list[i])
                break
            i+=1
        x+=1
    #Make a dictionary with the Airport code as the key and the value being the airport object of that key
    finalAirports = {}
    for i in airport_list:
        finalAirports[i[0]] = Airport(i[0], i[2], i[3], i[5])
    return finalAirports
        
        

## Building all the airplanes

In [267]:
#buildPlanes will look the aircraft csv and build a dictionary of planes, with airplane model as the 
#key and the values being the model, manufacturer, and range (converted to metric)

#time complexity: o(n)
def buildPlanes():
    aircraftDict = {}
    with open('aircraft.csv', newline='', encoding="utf8") as airplane_file: # opens the csv file
        reader = csv.reader(airplane_file) # reads the cotents to a variable
        next(reader, None) # returns none at the end of the file
        for airplane in reader: # iterates through the reader
            if airplane[2] == "imperial":
                airRange = int(airplane[4]) * 1.609
            else:
                airRange = airplane[4]
            aircraftDict[airplane[0]] = Aircraft(airplane[0], airplane[3], airRange)
    return aircraftDict
    
    

In [254]:
#This is a work in progress, my idea was that if a leg can't be flown, you remove it from the dictionary 
#of legs and distances. However, this will mean that the route calculator must also have those legs removed,
#So it will be complicated. Should try and figure out how to solve this

#Time complexity: o(n)
def checkPlaneAllowed(dictAirplane, routeDict, input_list):
    planeToFly = input_list[5]
    planeRange = dictAirplane[planeToFly].get_max_capacity()
    routeDict_copy = routeDict.copy()
    for j in routeDict_copy:
        if routeDict_copy[j] > planeRange:
            routeDict.pop(j)
    
    

## Distance

### Calculating the permutations

In [255]:
#Takes in the input list of airports and plane, also the dictionary of airport objects. Creates
#a permutation list from the input list of airports, then matches the airport name to its object.
#Uses matching and a bit more work to create each possible leg to be traveled from the permutations,
#and calculates the distance for each leg. 

#Time complexity: n^4
def permutations_no(listx, airport_list):
    w = listx[1]
    x = listx[2]
    y = listx[3]
    z = listx[4]
    a = listx[0]
    count = 0
    permlist = []
    perm = permutations([w,x,y,z]) 
    for i in list(perm): 
        permlist.append(i)

    airport_distances = {}
    
    for perms in permlist:
        perms = [a] + list(perms) + [a]
        for i in range(0, len(perms) - 1):
            for j in airport_list:
                if j == perms[i]:
                    airport1 = airport_list[perms[i]]
                    for k in airport_list:
                        if k == perms[i+1]:
                            airport2 = airport_list[perms[i+1]]
                    distance = distanceBetweenAirports(float(airport1.get_longitude()), float(airport1.get_latitude()), float(airport2.get_longitude()), float(airport2.get_latitude()))
                    if distance != 0:
                        airport_distances['_'.join([(airport1.airport_code),(airport2.airport_code)])]  = distance

                
    return airport_distances

In [256]:
#This will create permutations of all possible routes using the input list of airports and plane

#time complexity: o(n)

def allPerms(listx):
    w = listx[1]
    x = listx[2]
    y = listx[3]
    z = listx[4]
    a = listx[0]
    count = 0
    permlist = []
    newpermlist = []
    perm = permutations([w,x,y,z]) 
    for i in list(perm): 
        permlist.append(i)

    for perms in permlist:
        perms = [a] + list(perms) + [a]
        newpermlist.append(perms)

    return newpermlist

### Calculating the distance

In [257]:
def distanceBetweenAirports(latitude1,longitude1,latitude2,longitude2):
    radius_earth = 6371  # km
    theta1 = longitude1 * (2 * pi) / 360
    theta2 = longitude2 * (2 * pi) / 360
    phi1 = (90 - latitude1) * (2 * pi) / 360
    phi2 = (90 - latitude2) * (2 * pi) / 360
    distance = acos( sin(phi1) * sin(phi2) * cos(abs(theta1 - theta2)) +  cos(phi1) * cos(phi2) ) * radius_earth
    return floor(distance)

## Analysis

In [273]:
#from pprint import pprint
inputList = ['DUB', 'LHR', 'CPH', 'HEL', 'MOW', 'SAH']
airport_objects_dict = optimum_route(input_list)
#for i in airports.values():
#    pprint(vars(i))
#airports['CPH']
all_routes_list = allPerms(input_list)
dict_routes_distances = permutations_no(input_list, airports)
dictOfAirplanes = buildPlanes()






## Costing

In [274]:
#Goes through the dict of airport objects and the dictionary of legs (keys) and their distances(values).
#Converts costs into distance based on home airport (first 3 characters of leg), then multiplies by that 
#airports currency conversion to euros, and build a new dictionary with leg as the key and cost as value.

#time complexity: o(n^2)
def findCosts(leg_distance_dict, airport_object_dict):
    costDict = {}
    myKey = ""
    for i in leg_distance_dict:
        myKey = i[:3]
        x = 0
        cost = 0
        for j in airport_object_dict:
            if myKey == j:
                cost = round(float(airport_object_dict[j]._exchange_rate) * float(leg_distance_dict[i]), 2)
                costDict[i] = cost
            x+=1
    return costDict    
        

In [275]:
costs = findCosts(dict_routes_distances, airport_objects_dict)

## Costs and legs shenanigans

In [276]:
#This function will go through the lists of routes and the dictionary of costs for each leg of the route,
#Match the leg within each route and add up the costs for the route. A final dictionary will be created 
#with the key being the tuple containing the route, and the value the integer cost for the route. 

#time_complexity: o(n^2)
def findRouteCost(myList, costDict):
    routeCostDict = {}
    cost = 0
    x = 0
    
    while x < len(myList):
        i = 0
        cost = 0
        while i < (len(myList[x]) - 1):
            myKey = str(myList[x][i]) + "_" + str(myList[x][i+1])
            cost += costDict[myKey]
            cost = round(cost,2)
            i+=1
        routeCostDict[tuple(myList[x])] = cost
        x+=1
    return routeCostDict

In [278]:
findRouteCost(all_routes_list, costs)

{('DUB', 'LHR', 'CPH', 'HEL', 'MOW', 'DUB'): 2881.5,
 ('DUB', 'LHR', 'CPH', 'MOW', 'HEL', 'DUB'): 4066.63,
 ('DUB', 'LHR', 'HEL', 'CPH', 'MOW', 'DUB'): 4182.19,
 ('DUB', 'LHR', 'HEL', 'MOW', 'CPH', 'DUB'): 4127.16,
 ('DUB', 'LHR', 'MOW', 'CPH', 'HEL', 'DUB'): 6152.35,
 ('DUB', 'LHR', 'MOW', 'HEL', 'CPH', 'DUB'): 5058.09,
 ('DUB', 'CPH', 'LHR', 'HEL', 'MOW', 'DUB'): 4903.88,
 ('DUB', 'CPH', 'LHR', 'MOW', 'HEL', 'DUB'): 6946.99,
 ('DUB', 'CPH', 'HEL', 'LHR', 'MOW', 'DUB'): 6788.17,
 ('DUB', 'CPH', 'HEL', 'MOW', 'LHR', 'DUB'): 2925.47,
 ('DUB', 'CPH', 'MOW', 'LHR', 'HEL', 'DUB'): 6102.1,
 ('DUB', 'CPH', 'MOW', 'HEL', 'LHR', 'DUB'): 3938.69,
 ('DUB', 'HEL', 'LHR', 'CPH', 'MOW', 'DUB'): 5494.47,
 ('DUB', 'HEL', 'LHR', 'MOW', 'CPH', 'DUB'): 7598.11,
 ('DUB', 'HEL', 'CPH', 'LHR', 'MOW', 'DUB'): 6626.83,
 ('DUB', 'HEL', 'CPH', 'MOW', 'LHR', 'DUB'): 3790.44,
 ('DUB', 'HEL', 'MOW', 'LHR', 'CPH', 'DUB'): 4499.17,
 ('DUB', 'HEL', 'MOW', 'CPH', 'LHR', 'DUB'): 3704.4,
 ('DUB', 'MOW', 'LHR', 'CPH', '